# K-Armed Bandit Problems 

First notebook of the series on Reinforcement Learning (RL). This notebook (along with the rest of the reinforcement learning series) is based on [this introductory book](http://incompleteideas.net/book/bookdraft2018jan1.pdf) by R.S. Sutton and A.G Barto and on this [coursera cource](https://www.coursera.org/learn/fundamentals-of-reinforcement-learning/home/welcome). The reason I created these notebooks is mainly to gain a better understanding of RL myself and to try and implement the theory and models with python.

### Load the required libraries

In [1]:
import numpy as np
import gym
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from tqdm import tqdm 
from gym import spaces
from gym.utils import seeding

import time 

In [2]:
%matplotlib notebook

## What are the K-Armed Bandits?

In K-Armed Bandits there is one ***state** and k-**actions** for an **agent** to interact within an **environment***. Ecery action provides a (random) reward with an unknown value. The goal of the agent is to maximise the reward in the long run and to do so, the agent needs to explore all the actions and find the one that generates the highest reward. 

Based on the definition above, I will start building the structure of the environment using the **OpenAI**'s gym interface. The blocks of code below are highly influenced by [this code](https://github.com/diegoalejogm/openai-k-armed-bandits/tree/master/gym_armed_bandits).

In [3]:
m = np.array([[5, 1, 0, -10]]) # means for a 4-armed bandit 
sd = np.array([[1, 0.1, 5, 1]]) # standard deviations for a 4-armed bandit 
bandit_len = len(m[0]) # number of bandits 
# a = np.array([[1]]) # actions for a 4-armed bandit, play around with indecies: 0-3

## Building the structure of the environment 

The environment will receive ```np.array()``` for means and standard deviations for each action with dimensions ```num_experiments x num_bandits```. For the 4-armed bandit the action indecies should be from 0-3 (one at a time). 

### sample from the specified bandit and take steps. 

The size of the vector for each step is of size ```num_experiments```, where the action to take is specified for each experiment. As a return we get ```r``` which is the reward for each experiment/action 

In [45]:
class BanditsEnv():
    
    '''Here, we get an array of length k. Each item is a function which samples 
    from a specified distribution.'''
    
    def __init__(self, m, sd):
        
        assert len(m.shape) == 2
        assert len(sd.shape) == 2
        
        super().__init__()
        
        # define action and obs spaces
        self.mean = m
        self.sd = sd
        self.nb_bandits = len(m[0]) # number of bandits 
        self.nb_exp = len(m) # number of experiments 
        self.action_space = spaces.Discrete(self.nb_bandits) # define action space
        self.obs_space = spaces.Discrete(1) # define observetional space
    
    # define the step function  
    def step(self, a):
        
        # input arg: array that contains the current action 
        # output: reward array for the current action 
        
        assert (a < self.nb_bandits).all()
        
        sampled_m = self.mean[np.arange(self.nb_exp), a]
        sampled_sd = self.sd[np.arange(self.nb_exp), a]
        r = np.random.normal(loc=sampled_m, scale=sampled_sd, size=(self.nb_exp, self.nb_exp))
        
        # for now return only reward 
        return r      

### Creating and interacting with the environment 
Now let's create an environment by passing arrays of the means and standard deviations as input arguments. Every time we interact with the environment we specify which bandid we want to pull, and the environment will return the associated reward

In [47]:
# create an environment 
test_env = BanditsEnv(m, sd)
rewards = np.zeros(m.shape) # we'll store the rewards here

# get the reward for each action
for i in range(4):
    a = np.array([[i]])
    r = test_env.step(a)
    rewards[0,i] = r
    print(" reward for bandit", i+1, "was:", r[0])

 reward for bandit 1 was: [3.3710695]
 reward for bandit 2 was: [1.04975842]
 reward for bandit 3 was: [3.61570152]
 reward for bandit 4 was: [-10.9507265]


Above I created 4 bandits (with their means and sd). I interacted with each bandit to see the reward it returns. The reward looks random, however it seems that it falls near the mean of each bandit. The sd shows how far the rewards will fall for each bandit. For example, the deviation for the second bandit is very small (0.1) and we see that the reward is very close to the mean. The sd of the third bandit is high (5) and it seems that the reward always falls quit far from the mean.

#### Exercise and Practice 
This example has 4 bandits. Play around with the Environment and try running it with more bandits (e.g. 6 or 10) or change the means and standard deviations.

### The exploration - Explotation Dilemma
Note that interacting with each bandit just once doesn't tell us much. The goal in such tasks is to interact with the environment many times, explore all actions/bandits and build a distribution of rewards for each action/bandit, in an attempt to choose an action that gives the highest reward.

This notion is very familiar in situations from real life. Imagine being at a restaurant and trying to choose between ordering a meal that you have tried before and you know is good (previous high reward) or ordering a meal that is new but looks delicious. What should you choose? You know that the familiar meal will give you a high reward, but what if the new meal is even tastier? 

Choosing the familiar meal is a greedy behaviour because you already know that this choice will give you high reward (exploitation), however, it stops you from trying other options (exploration). This is called the exploration-explotation dilemma and is a very common problem in k-armed bandit tasks. The dilemma is presented every time the agent is in an **uknown or partially known** environment and trys to optimise its interaction with the environment. The agent wants to get the highest possible reward but does not know which action is the best for achieving this goal. By exploring (trying different actions or trying different meals every time we go to a specific restaurant), we interact with the environment, we learn more about it but we also sacrifice the chance of getting a known (high reward). This may look sub-optimal, but, **in the long run** new actions may give higher rewards than we initially thought.

This dilemma is very common in RL situations, thus developing good strategies for dealing with it is very important.

Before developing the strategies we should first see how we can evaluate our actions. 

### Evaluating actions

Now that we have developed our environment, let us evaluate our actions. For each of the 4 arms we need a value of what we expect to get from the given action. In the beginning, we do not know what to expect, however, every time we interact with the environment (every time we take an action) we update our expectations.

I will start by pulling the first arm 5 times. For this behaviour, I will receive a list of rewards:

In [50]:
eval_env = BanditsEnv(m, sd)
first_arm = []

# get the reward for each action 5 times (5 pulls) and store the values for the first arm in a list
for i in range (5):
    rewards = np.zeros(m.shape) # for every pull, we'll store the rewards here
    
    for j in range(4):
        a = np.array([[j]])
        r = eval_env.step(a)
        rewards[0,j] = r
        
    first_arm.append(rewards[0,0]) # store the actions for the 1st arm

first_arm # visualise     

[3.2614877253822656,
 4.451492078087157,
 3.556804178285624,
 4.408528829420389,
 5.958797910455155]